# Track C

In [ ]:
import os, networkx as nx
import pickle

print(os.listdir("./data_week6"))
with open("./data_week6/graph_stock_forecast.gpickle", "rb") as f:
    G = pickle.load(f)
print("✅ Graph OK →", len(G.nodes()), "nodes,", len(G.edges()), "edges")

['graph_stock_forecast.gpickle', 'ablation_results_graph.csv', 'corpus.csv']
✅ Graph OK → 19 nodes, 20 edges


In [ ]:
%%writefile app.py
# ======================================================
# Week 6 – Track C: Streamlit Multi-Hop Graph-RAG Demo
# ======================================================
import streamlit as st
import networkx as nx
import matplotlib.pyplot as plt
import re
from pathlib import Path
import pickle

# ---------- Load your real graph ----------
DATA_DIR = Path("./data_week6")
GRAPH_PATH = DATA_DIR / "graph_stock_forecast.gpickle"

if GRAPH_PATH.exists():
    with open(GRAPH_PATH, "rb") as f:
        G = pickle.load(f)
    print(f"✅ Loaded real graph with {len(G.nodes())} nodes and {len(G.edges())} edges.")
else:
    print("⚠️ Could not find graph_stock_forecast.gpickle in ./data_week6/. "
          "Run Track A to create it.")
    G = nx.Graph()  # empty placeholder

# ---------- Track B functions (inline) ----------
def decompose(query: str):
    q = query.lower()
    if "agent" in q and "model" in q:
        return ["Which agent integrates sentiment and market indicators?",
                "Which model does that agent use?"]
    elif "agent" in q and "dataset" in q:
        return ["Which agent analyzes financial sentiment?",
                "Which datasets were used by that agent?"]
    elif "sentiment-agent" in q and "removed" in q:
        return ["What happens when the sentiment-agent is removed?",
                "Which metric decreases as a result?"]
    elif "accuracy" in q or "rmse" in q:
        return ["Which model achieved the best accuracy or RMSE?",
                "Which agent used that model?"]
    return [query]

def neighbors_for(node):
    spans = []
    if node not in G: return spans
    for u, v, data in G.edges(node, data=True):
        spans.append({
            "u": u, "v": v,
            "doc_id": data.get("doc_id"),
            "sentence": data.get("sentence"),
            "relation": data.get("relation", "related_to")
        })
    return spans

def answer_subq(subq, memory):
    q = subq.lower()
    spans = []

    # Hop 1: find agent by clues in sentences
    if "agent" in q and ("sentiment" in q or "market" in q or "indicator" in q):
        for node in G.nodes():
            if "Agent" in node:
                for ev in neighbors_for(node):
                    s = ev["sentence"].lower()
                    if "sentiment" in s and ("indicator" in s or "market" in s):
                        return {"subq": subq, "answer": node, "evidence": [ev]}
    elif "agent" in q and "sentiment" in q:
        for node in G.nodes():
            if "Agent" in node:
                for ev in neighbors_for(node):
                    if "sentiment" in ev["sentence"].lower():
                        return {"subq": subq, "answer": node, "evidence": [ev]}

    # Hop 2: agent → model
    if "model" in q and "agent" in q:
        for node in G.nodes():
            if "Agent" in node:
                for ev in neighbors_for(node):
                    m = re.search(r"FinBERT|Transformer|LSTM|ARIMA|Fusion", ev["sentence"], re.I)
                    if m:
                        return {"subq": subq, "answer": m.group(0), "evidence": [ev]}

    # Hop 2: agent → dataset
    if ("dataset" in q or "train" in q) and "agent" in q:
        for node in G.nodes():
            if "Agent" in node:
                for ev in neighbors_for(node):
                    d = re.search(r"NASDAQ|NYSE|Twitter|Reddit|financial\s+news", ev["sentence"], re.I)
                    if d:
                        return {"subq": subq, "answer": d.group(0), "evidence": [ev]}

    # Metric / performance
    if any(k in q for k in ["metric", "accuracy", "rmse", "f1", "auc"]):
        for node in G.nodes():
            if re.search(r"Accuracy|RMSE|F1|AUC", str(node), re.I):
                spans = neighbors_for(node)
                return {"subq": subq, "answer": node, "evidence": spans}

    # Ablation
    if "sentiment-agent" in q and "removed" in q:
        for node in G.nodes():
            if "Agent Alpha" in node or "Sentiment" in node:
                for ev in neighbors_for(node):
                    s = ev["sentence"].lower()
                    if "decrease" in s or "reduction" in s:
                        return {"subq": subq, "answer": "Performance decreased (≈ 9% on F1)", "evidence": [ev]}

    # Fallback: return some agent evidence if possible
    if not spans and "agent" in q:
        for node in G.nodes():
            if "Agent" in node:
                spans = neighbors_for(node)
                if spans:
                    return {"subq": subq, "answer": node, "evidence": spans}

    return {"subq": subq, "answer": "No direct evidence found", "evidence": spans}

def multi_hop(query):
    subs = decompose(query)
    memory, hops = {}, []
    for s in subs:
        h = answer_subq(s, memory)
        hops.append(h)
        memory[len(hops)] = h["answer"]
    final = " ; ".join(h["answer"] for h in hops)
    citations = sorted({ev["doc_id"] for h in hops for ev in h["evidence"] if "doc_id" in ev})
    return {"query": query, "subqs": subs, "hops": hops, "final": final, "citations": citations}

# ---------- Streamlit UI ----------
st.set_page_config(page_title="Multi-Hop Graph-RAG", layout="wide")
st.title("🧠 Multi-Hop Graph-RAG for Stock Forecasting")

st.sidebar.header("⚙️ Controls")
st.sidebar.markdown(f"**Graph:** {len(G.nodes())} nodes • {len(G.edges())} edges")

query = st.text_input(
    "Enter your question:",
    "Which model does the agent that integrates sentiment and market indicators use?"
)

if st.button("Run Query"):
    with st.spinner("Running multi-hop reasoning..."):
        result = multi_hop(query)

    st.subheader("✅ Final Answer")
    st.markdown(f"**{result['final']}**")
    if result["citations"]:
        st.caption(f"Citations: {', '.join(result['citations'])}")

    st.subheader("🔍 Reasoning Trace")
    for i, hop in enumerate(result["hops"], 1):
        with st.expander(f"Hop {i}: {hop['subq']} → {hop['answer']}"):
            for ev in hop["evidence"][:3]:
                st.write(f"- ({ev['doc_id']}) {ev['sentence']}")

    # Graph neighborhood of evidence
    sub_nodes = set()
    for hop in result["hops"]:
        for ev in hop["evidence"]:
            sub_nodes.update([ev["u"], ev["v"]])
    if sub_nodes:
        subgraph = G.subgraph(sub_nodes)
        fig, ax = plt.subplots(figsize=(6, 4))
        nx.draw_networkx(subgraph, with_labels=True, node_color="lightblue", font_size=8, ax=ax)
        st.pyplot(fig)
else:
    st.info("💡 Type a question and click **Run Query**.")


Overwriting app.py


In [ ]:
!pip install -q streamlit
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb >/dev/null
!fuser -k 8501/tcp || true
!streamlit run app.py --server.port 8501 &>/dev/null&
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate


2025-10-06T22:43:40Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-06T22:43:40Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-06T22:43:43Z INF +--------------------------------------------------------------------------------------------+
2025-10-06T22:43:43Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-06T22:43:43Z INF |  https://tracy-representation-geo-concentrate.trycloud